In [23]:
import os
import json
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to your dataset directories
train_dir = 'train'
validation_dir = 'valid'
test_dir = 'test'

# Load COCO JSON files
def load_annotations(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)['annotations']
    return data

train_annotations = load_annotations('train/_annotations.coco.json')
valid_annotations = load_annotations('valid/_annotations.coco.json')
test_annotations = load_annotations('test/_annotations.coco.json')

# Set up data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')


Found 324 images belonging to 2 classes.
Found 30 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16

# Load the VGG16 model pretrained on ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
base_model.trainable = False

# Add custom classification layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')  # Adjust output classes as per your dataset
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy}')




58889256/58889256 [==============================] - 7s 0us/step

Epoch 1/10


11/11 [==============================] - 31s 3s/step - loss: 2.9420 - accuracy: 0.4784 - val_loss: 0.7628 - val_accuracy: 0.4667
Epoch 2/10
11/11 [==============================] - 32s 3s/step - loss: 0.6547 - accuracy: 0.5957 - val_loss: 0.7669 - val_accuracy: 0.5333
Epoch 3/10
11/11 [==============================] - 33s 3s/step - loss: 0.5514 - accuracy: 0.6821 - val_loss: 0.4824 - val_accuracy: 0.8000
Epoch 4/10
11/11 [==============================] - 41s 4s/step - loss: 0.3876 - accuracy: 0.8519 - val_loss: 0.4393 - val_accuracy: 0.7333
Epoch 5/10
11/11 [==============================] - 44s 4s/step - loss: 0.3123 - accuracy: 0.8889 - val_loss: 0.4149 - val_accuracy: 0.8000
Epoch 6/10
11/11 [==============================] - 42s 4s/step - loss: 0.2662 - accuracy: 0.9198 - val_loss: 0.4012 - val_accuracy: 0.7667
Epoch 7/10
11/11 [==============================] - 44s 4s/step - loss: 0.2250 - accuracy:

In [27]:
model.save('Model_VGG16')

INFO:tensorflow:Assets written to: Model_VGG16\assets


INFO:tensorflow:Assets written to: Model_VGG16\assets


In [32]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('C:/Users/SANIKA/Desktop/Research Intern/Melanoma_classification.v1i.coco/Model_VGG16')

# Define a function to preprocess the image and predict the class
def predict_image_class(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0  # Normalize pixel values
    pred = model.predict(x)
    class_index = np.argmax(pred)
    return class_index

# Example usage to predict an external image
image_path = 'xyz.jpg'  # Replace with the path to your image
predicted_class_index = predict_image_class(image_path, model)

# Print the predicted class index
print("Predicted Class Index:", predicted_class_index)


1/1 [==============================] - 0s 257ms/step
Predicted Class Index: 1
